In [46]:
import requests
from bs4 import BeautifulSoup
from os.path  import basename
import time as t
from random import randrange as rr
import json
import numpy as np
import pandas as pd
import hashlib

In [47]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

In [48]:
# season_list = [x for x in range(2010, 2019)]
# season_list

In [49]:
# season_list = [x for x in range(2010, 2019)]

# for season in season_list:
#     page = 'https://www.transfermarkt.at/bundesliga/startseite/wettbewerb/A1/plus/?saison_id=' + str(season)

#     # beginn der Zeitmessung und übergabe des html dokuments
#     start = t.time()
#     tree = requests.get(page, headers = headers)
#     soup = BeautifulSoup(tree.content, 'html.parser')
    
#     club_list = []


#     for i in range(len(soup.select('a.vereinprofil_tooltip'))):
#         try:
#             club_list.append(soup.select('a.vereinprofil_tooltip')[i].attrs['id'])
#         except: KeyError

#     club_list = list(set(club_list))
#     print(club_list)
#     t.sleep(3)

In [50]:
# club_list = []


# for i in range(len(soup.select('a.vereinprofil_tooltip'))):
#     try:
#         club_list.append(soup.select('a.vereinprofil_tooltip')[i].attrs['id'])
#     except: KeyError
        
# club_list = list(set(club_list))
# print(club_list)

In [51]:
# page = 'https://www.transfermarkt.at/red-bull-salzburg/spielplan/verein/2662/saison_id/2018/plus/1'


# # beginn der Zeitmessung und übergabe des html dokuments
# start = t.time()
# tree = requests.get(page, headers = headers)
# soup = BeautifulSoup(tree.content, 'html.parser')


In [52]:
game_list = []
season_list = [x for x in range(2009, 2019)]


# for season in season_list:
#     print(season)
#     t.sleep(1)
season = 2019
page_league = 'https://www.transfermarkt.at/bundesliga/startseite/wettbewerb/A1/plus/?saison_id=' + str(season)

tree = requests.get(page_league, headers = headers)
soup = BeautifulSoup(tree.content, 'html.parser')

club_list = []


for i in range(len(soup.select('a.vereinprofil_tooltip'))):
    try:
        club_list.append(soup.select('a.vereinprofil_tooltip')[i].attrs['id'])
    except: KeyError

club_list = list(set(club_list))


for club in club_list:
    print(club)
    t.sleep(3)
    page_club = 'https://www.transfermarkt.at/red-bull-salzburg/spielplan/verein/' + str(club) + '/saison_id/' + str(season) + '/plus/1'
    # beginn der Zeitmessung und übergabe des html dokuments
    start = t.time()
    tree = requests.get(page_club, headers = headers)
    soup = BeautifulSoup(tree.content, 'html.parser')


    for i in range(4, len(soup.select('div.box'))):
        box = soup.select('div.box')[i]
        competition = soup.select('div.box')[i]('div')[0]('img')[0].attrs['alt']
        saison = str(season) + '/' + str(int(str(season)[-2:])+1)

        for j in range(1, len(box.select('tr'))):
            line = box.select('tr')[j]

            gameday = line.select('td')[0].get_text().strip()
            date = line.select('td')[1].get_text()
            time = line.select('td')[2].get_text()
            home_team_id = line.select('td')[3]('a')[0].attrs['id']
            home_team_name = line.select('td')[3]('img')[0].attrs['alt']
            guest_team_id = line.select('td')[5]('a')[0].attrs['id']
            guest_team_name = line.select('td')[5]('img')[0].attrs['alt']

            if club == home_team_id:
                home_coach = line.select('td')[8].get_text()
                guest_coach = ''
                home_game_system = line.select('td')[7].get_text()
                guest_game_system = ''
            else:
                home_coach = ''
                guest_coach = line.select('td')[8].get_text()
                home_game_system = ''
                guest_game_system = line.select('td')[7].get_text()

            try:
                viewers = int(line.select('td')[9].get_text().replace('.', ''))
            except:
                viewers = 0
            result = line.select('td')[10].get_text().strip()

            hash = hashlib.sha256()
            hash.update(bytes(str(viewers), 'utf-8'))
            hash.update(bytes(str(result), 'utf-8'))
            hash.update(bytes(str(date), 'utf-8'))
            hash.update(bytes(str(time), 'utf-8'))

            game_list.append({
                'saison': saison,
                'competition': competition,
                'gameday': gameday,
                'date': date,
                'time': time,
                'home_team_id': home_team_id,
                'home_team_name': home_team_name,
                'guest_team_id': guest_team_id,
                'guest_team_name': guest_team_name,
                'home_game_system': home_game_system,
                'guest_game_system': guest_game_system,
                'home_coach': home_coach,
                'guest_coach': guest_coach,
                'viewers': viewers,
                'result': result,
                "identifier": hash.hexdigest()
            })

            #print(game_list[-1])

413
2446
3551
533
14
503
409
2662
4467
4441
856
122
170


In [53]:
game_list[0], len(game_list)

({'saison': '2019/20',
  'competition': 'UEFA Champions League-Qualifikation',
  'gameday': '3.Runde Hinspiele',
  'date': 'Mi. 07.08.2019',
  'time': '20:00',
  'home_team_id': '26',
  'home_team_name': 'FC Basel 1893',
  'guest_team_id': '413',
  'guest_team_name': 'LASK',
  'home_game_system': '',
  'guest_game_system': '3-4-3 flach',
  'home_coach': '',
  'guest_coach': 'Valérien Ismaël',
  'viewers': 20470,
  'result': '1:2',
  'identifier': '090fd19b5b696a68fc0a37294f0eb6be442a189c4ea3b22f14bb02542245b13a'},
 296)

In [54]:
df = pd.DataFrame(game_list)

In [55]:
df.head()

,competition,date,gameday,guest_coach,guest_game_system,guest_team_id,guest_team_name,home_coach,home_game_system,home_team_id,home_team_name,identifier,result,saison,time,viewers
0,UEFA Champions League-Qualifikation,Mi. 07.08.2019,3.Runde Hinspiele,Valérien Ismaël,3-4-3 flach,413,LASK,,,26,FC Basel 1893,090fd19b5b696a68fc0a37294f0eb6be442a189c4ea3b2...,1:2,2019/20,20:00,20470
1,UEFA Champions League-Qualifikation,Di. 13.08.2019,3.Runde Rückspiele,,,26,FC Basel 1893,Valérien Ismaël,3-4-3 flach,413,LASK,61a29314c1f03bbc31f67e84ff679cf4dd5d6d72ae14d9...,3:1,2019/20,20:30,14000
2,UEFA Champions League-Qualifikation,Di. 20.08.2019,Qualifikationsrunde - Hinspiele,,,2282,FC Brügge,,,413,LASK,fa118ca2c8e433414495b7514b87c7d671dca40feb4744...,-:-,2019/20,21:00,0
3,UEFA Champions League-Qualifikation,Mi. 28.08.2019,Qualifikationsrunde - Rückspiele,,,413,LASK,,,2282,FC Brügge,f266fa0ac1906590f1f659cb521c2130358bb0e184fc32...,-:-,2019/20,-,0
4,Bundesliga,So. 28.07.2019,1,,,3551,SC Rheindorf Altach,Valérien Ismaël,3-4-3 flach,413,LASK,b6bb3e2fe81cb2a977fbe7669384ce4cd4a0cdd6f2cc75...,2:0,2019/20,17:00,5333


In [56]:
df.tail()

,competition,date,gameday,guest_coach,guest_game_system,guest_team_id,guest_team_name,home_coach,home_game_system,home_team_id,home_team_name,identifier,result,saison,time,viewers
291,Bundesliga,Sa. 22.02.2020,20,,,170,SK Rapid Wien,,,4467,TSV Hartberg,87700720a3a9a35297b237099fdf11ce27ede0c3ebf926...,-:-,2019/20,-,0
292,Bundesliga,Sa. 29.02.2020,21,,,856,SV Mattersburg,,,170,SK Rapid Wien,16f28d70f833486bc5e0c35888d8c08bd913ea0b0831a9...,-:-,2019/20,-,0
293,Bundesliga,Sa. 07.03.2020,22,,,170,SK Rapid Wien,,,4441,Wolfsberger AC,ffb0b5b6f91012eb40b40f6fe4d979084566905e179e10...,-:-,2019/20,-,0
294,ÖFB-Cup,So. 21.07.2019,1.Runde,Dietmar Kühbauer,3-5-2,170,SK Rapid Wien,,,10281,SV Allerheiligen,9fec2e29c0f745ff082828358d1a6ec353a92117d17ce9...,1:9,2019/20,17:15,2200
295,ÖFB-Cup,Mi. 25.09.2019,2.Runde,,,409,Red Bull Salzburg,,,170,SK Rapid Wien,da5a424a25775d57cf1f178a0ab807e7d4ddcad34da675...,-:-,2019/20,20:45,0


In [57]:
df[df.result.str.len() > 3]

,competition,date,gameday,guest_coach,guest_game_system,guest_team_id,guest_team_name,home_coach,home_game_system,home_team_id,home_team_name,identifier,result,saison,time,viewers
76,DFB-Pokal,Sa. 10.08.2019,1.Runde,Alfred Schreuder,3-4-1-2,533,TSG 1899 Hoffenheim,,,1557,Würzburger Kickers,172e9256d7fc872af69a3a42369fcd6ced39f717a0b15b...,7:8 n.E.,2019/20,18:30,13090
197,ÖFB-Cup,Sa. 20.07.2019,1.Runde,Markus Schopp,4-2-3-1,4467,TSV Hartberg,,,18517,SC Wiener Viktoria,6476a553ed4a52e8c3b1ac99b74771d628c0c520c1b900...,7:5 n.E.,2019/20,17:30,500
244,ÖFB-Cup,Sa. 20.07.2019,1.Runde,Franz Ponweiser,3-5-2,856,SV Mattersburg,,,16802,SC Pinkafeld,0fd10beeb67e945d23ef51c47a619920477b7fff6480f0...,0:10,2019/20,17:30,650


In [58]:
league = page_league.split('/')[3]
season_start = min(season_list)
season_end = max(season_list)
file_name = league + '-' + str(season_start) + '-' + str(season_end)
file_name

'bundesliga-2009-2018'

In [59]:
df.to_csv(file_name + 'erste.csv')

16